<a href="https://colab.research.google.com/github/magdachmiel/dw_matrix/blob/master/Transformacja%233/Day5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!pip install hyperopt

## Optymalizacja parametrów z hyperopt


In [0]:
import pandas as pd
import numpy as np
import os
import datetime

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPool2D, Dense, Flatten, Dropout
from tensorflow.keras.utils import to_categorical

%load_ext tensorboard

import matplotlib.pyplot as plt
from skimage import color, exposure

from sklearn.metrics import accuracy_score

from hyperopt import hp, STATUS_OK, tpe, Trials, fmin

In [0]:
cd '/content/drive/My Drive/Colab Notebooks/dw_matrix/Transformacja#3'

/content/drive/My Drive/Colab Notebooks/dw_matrix/Transformacja#3


In [0]:
train = pd.read_pickle('data/train.p')
X_train, y_train = train['features'], train['labels']

test = pd.read_pickle('data/test.p')
X_test, y_test = test['features'], test['labels']

In [0]:
if y_train.ndim == 1: y_train = to_categorical(y_train)
if y_test.ndim == 1: y_test = to_categorical(y_test)

In [0]:
input_shape = X_train.shape[1:]
num_classes = y_train.shape[1]

In [0]:
def train_model(model, X_train, y_train, params_fit={}):
  
  model.compile(loss='categorical_crossentropy', optimizer='Adam', metrics=['accuracy'])

  logdir = os.path.join('logs', datetime.datetime.now().strftime('%Y%m%d-%H%M%S'))
  tensorboard_callback = tf.keras.callbacks.TensorBoard(logdir, histogram_freq=1)

  model.fit(
      X_train, 
      y_train,
      batch_size = params_fit.get('batch_size', 128),
      epochs = params_fit.get('epochs', 5),
      verbose = params_fit.get('verbose', 1),
      validation_data = params_fit.get('validation_data', (X_train, y_train)),
      callbacks = [tensorboard_callback]
  )

  return model


In [0]:
def predict(model_trained, X_test, y_test, scoring = accuracy_score):
  y_pred_prob = model_trained.predict(X_test)
  y_pred = np.argmax(y_pred_prob, axis=1)

  y_test_norm = np.argmax(y_test, axis=1) #znormalizowany wektor wynikow ze zbioru testowego

  return accuracy_score(y_test_norm, y_pred)

In [0]:
def train_and_predict(model): 
  model_trained = train_model(model, X_train, y_train)
  return predict(model_trained, X_test, y_test)

In [0]:
def get_cnn_v5(input_shape, num_classes):
  return Sequential([
  Conv2D(filters=32, kernel_size=(3,3), activation='relu', input_shape=input_shape),
  Conv2D(filters=32, kernel_size=(3,3), activation='relu', padding='same'),
  MaxPool2D(),
  Dropout(0.3),

  Conv2D(filters=64, kernel_size=(3,3), activation='relu', padding='same'),
  Conv2D(filters=64, kernel_size=(3,3), activation='relu'),
  MaxPool2D(),
  Dropout(0.3),

  Conv2D(filters=128, kernel_size=(3,3), activation='relu', padding='same'),
  Conv2D(filters=128, kernel_size=(3,3), activation='relu'),
  MaxPool2D(),
  Dropout(0.3),

  Flatten(),

  Dense(1024, activation='relu'),
  Dropout(0.3),

  Dense(1024, activation='relu'),
  Dropout(0.3),

  Dense(num_classes, activation='softmax')                
  ])

In [0]:
model_trained = train_model(get_cnn_v5(input_shape, num_classes), X_train, y_train)
predict_results = predict(model_trained, X_test, y_test)

Epoch 1/5
272/272 [==============================] - 4s 16ms/step - loss: 2.4558 - accuracy: 0.3410 - val_loss: 0.8395 - val_accuracy: 0.7690
Epoch 2/5
272/272 [==============================] - 4s 15ms/step - loss: 0.7879 - accuracy: 0.7514 - val_loss: 0.2045 - val_accuracy: 0.9391
Epoch 3/5
272/272 [==============================] - 4s 15ms/step - loss: 0.4146 - accuracy: 0.8709 - val_loss: 0.0953 - val_accuracy: 0.9722
Epoch 4/5
272/272 [==============================] - 4s 15ms/step - loss: 0.2839 - accuracy: 0.9130 - val_loss: 0.0554 - val_accuracy: 0.9844
Epoch 5/5
272/272 [==============================] - 4s 15ms/step - loss: 0.2018 - accuracy: 0.9399 - val_loss: 0.0319 - val_accuracy: 0.9906


In [0]:
model_trained.evaluate(X_test, y_test)

138/138 [==============================] - 0s 3ms/step - loss: 0.1317 - accuracy: 0.9644


[0.13165998458862305, 0.9643990993499756]

## Tworzenie funkcji celu do optymalizacji

In [0]:
def get_model(params):
  return Sequential([
  Conv2D(filters=32, kernel_size=(3,3), activation='relu', input_shape=input_shape),
  Conv2D(filters=32, kernel_size=(3,3), activation='relu', padding='same'),
  MaxPool2D(),
  Dropout(params['dropout_cnn_block_1']),

  Conv2D(filters=64, kernel_size=(3,3), activation='relu', padding='same'),
  Conv2D(filters=64, kernel_size=(3,3), activation='relu'),
  MaxPool2D(),
  Dropout(params['dropout_cnn_block_2']),

  Conv2D(filters=128, kernel_size=(3,3), activation='relu', padding='same'),
  Conv2D(filters=128, kernel_size=(3,3), activation='relu'),
  MaxPool2D(),
  Dropout(params['dropout_cnn_block_3']),

  Flatten(),

  Dense(1024, activation='relu'),
  Dropout(params['dropout_dense_block_4']),

  Dense(1024, activation='relu'),
  Dropout(params['dropout_dense_block_5']),

  Dense(num_classes, activation='softmax')                
  ])

In [0]:
def func_obj(params):

  #model = get_cnn_v5(input_shape, num_classes)
  model = get_model(params)

  model.compile(loss='categorical_crossentropy', optimizer='Adam', metrics=['accuracy'])

  model.fit(
      X_train, 
      y_train,
      batch_size = int(params.get('batch_size')),
      epochs = 10,
      verbose = 0
  )

  accuracy =  model.evaluate(X_test, y_test)[1]
  print(' For params = ', params, ' ACC = {}'.format(accuracy))

  return {'loss' : -accuracy, 'status' : STATUS_OK, 'model' : model}

In [0]:
space = {
    'batch_size' : hp.quniform('batch_size', 50, 200, 10),
    'dropout_cnn_block_1' : hp.quniform('dropout_cnn_block_1', 0.2, 0.5, 0.1),
    'dropout_cnn_block_2' : hp.quniform('dropout_cnn_block_2', 0.2, 0.5, 0.1),
    'dropout_cnn_block_3' : hp.quniform('dropout_cnn_block_3', 0.2, 0.5, 0.1),
    'dropout_dense_block_4' : hp.quniform('dropout_dense_block_4', 0.2, 0.7, 0.1),
    'dropout_dense_block_5' : hp.quniform('dropout_dense_block_5', 0.2, 0.7, 0.1),
}

best = fmin(
    func_obj,
    space,
    tpe.suggest,
    30,
    Trials()
)

  1/138 [..............................]
 - ETA: 0s - loss: 0.9057 - accuracy: 0.8125

 17/138 [==>...........................]
 - ETA: 0s - loss: 0.1507 - accuracy: 0.9577

 29/138 [=====>........................]
 - ETA: 0s - loss: 0.1916 - accuracy: 0.9429

 43/138 [========>.....................]
 - ETA: 0s - loss: 0.1340 - accuracy: 0.9600

 56/138 [===========>..................]
 - ETA: 0s - loss: 0.1667 - accuracy: 0.9509

 68/138 [=============>................]
 - ETA: 0s - loss: 0.1491 - accuracy: 0.9559


In [0]:
best

{'batch_size': 200.0,
 'dropout_cnn_block_1': 0.30000000000000004,
 'dropout_cnn_block_2': 0.2,
 'dropout_cnn_block_3': 0.5,
 'dropout_dense_block_4': 0.2,
 'dropout_dense_block_5': 0.6000000000000001}

## Najlepsze wyniki dla zadanej przestrzeni parametrów

Bez optymalizacji parametrów - loss: 0.2339 - accuracy: 0.9635

Z optymalizacją - - loss: 0.0894 - accuracy: 0.9785


## Weryfikacja modelu z parametrami

In [0]:
params = best

In [0]:
model_ver = get_model(params)

model_ver.compile(loss='categorical_crossentropy', optimizer='Adam', metrics=['accuracy'])

model_ver.fit(
      X_train, 
      y_train,
      batch_size = int(params.get('batch_size', 128)),
      epochs = 7,
      verbose = 0
)

accuracy =  model_ver.evaluate(X_test, y_test)[0]

138/138 [==============================] - 0s 3ms/step - loss: 0.0894 - accuracy: 0.9785
